# Managing Pydantic Data Models in FastAPI

```{admonition} Attribution
Notes on *Chapter 4: Managing Pydantic Data Models in FastAPI* of {cite}`Voron2021`. The source files for running the background local servers can be found [here](https://github.com/particle1331/machine-learning/tree/master/docs/notebooks/fastapi/src/chapter4).
```

This chapter will cover in more detail the definition of a data model with Pydantic, the 
underlying data validation library used by FastAPI. 

**Goals** 
   * Defining models and their field types with Pydantic
   * Creating model variations with class inheritance
   * Adding custom data validation with Pydantic
   * Working with Pydantic objects

## Defining models and their field types with Pydantic

Pydantic is a powerful library for defining data models using Python classes and type 
hints. This approach makes those classes completely compatible with static type checking.

### Standard field types

We'll begin by defining fields with standard types, which only involve simple type hints.

In [21]:
from datetime import date
from enum import Enum
from typing import List
from pydantic import BaseModel

class Gender(str, Enum):
    MALE = "male"
    FEMALE = "female"
    NON_BINARY = "non-binary"

class Address(BaseModel):
    street_address: str
    postal_code: str
    city: str
    country: str

class Person(BaseModel):
    id: int
    first_name: str
    last_name: str
    address: Address
    gender: Gender
    birthdate: date 
    interests: List[str]

Let's instantiate one example:

In [22]:
Person(
    id=3,
    first_name="Ron", 
    last_name="Medina", 
    address=Address(
        street_address="#1 Street",
        postal_code="333",
        city="City",
        country="Country"),
    gender=Gender.MALE,
    birthdate=date(2021, 7, 21), # YYYY-MM-DD
    interests=[],
)

Person(id=3, first_name='Ron', last_name='Medina', address=Address(street_address='#1 Street', postal_code='333', city='City', country='Country'), gender=<Gender.MALE: 'male'>, birthdate=datetime.date(2021, 7, 21), interests=[])

This is quite powerful, we can have quite complex field types. Moreover, the pydantic model performs automatic validation and type conversion:

In [23]:
Person(
    id="3",
    first_name="Ron", 
    last_name="Medina", 
    gender="male",
    birthdate="2021-07-21",
    interests=[],
    address={
        "street_address": "#1 Street",
        "postal_code": "333",
        "city": "City",
        "country": "Country",
    }
)

Person(id=3, first_name='Ron', last_name='Medina', address=Address(street_address='#1 Street', postal_code='333', city='City', country='Country'), gender=<Gender.MALE: 'male'>, birthdate=datetime.date(2021, 7, 21), interests=[])

### Optional fields and default values

Be careful: don't assign default values that
    are dynamic types such as datetimes. By doing so, the datetime instantiation will be 
    evaluated only once when the model is *imported*. The effect of this is that all the 
    objects you'll instantiate will then share the same value instead of having a fresh value. This is a known Python gotcha. 


In [38]:
import time 
from datetime import datetime
from typing import Optional

class Post(BaseModel):
    date_created: date = datetime.now()
    tag: Optional[str]


post1 = Post()
time.sleep(3)
post2 = Post()
print((post1.date_created - post2.date_created).total_seconds())

0.0


Fortunately, Pydantic provides a `Field` 
function that allows us to set some advanced options on our fields, including one to set 
a factory for creating dynamic values.

### Field validation

It turns out that the validation for request parameters come directly from Pydantic. The syntax is very similar to the one we saw for `Path`, `Query`, and `Body`.

In [30]:
from pydantic import BaseModel, Field, ValidationError

class Person(BaseModel):
    first_name: str = Field(..., min_length=3)
    last_name: str = Field(..., min_length=3)
    age: Optional[int] = Field(None, ge=0, le=120)

Let's trigger an error with first name of length zero.

In [31]:
p = Person(first_name="", last_name="Shorttail")

ValidationError: 1 validation error for Person
first_name
  ensure this value has at least 3 characters (type=value_error.any_str.min_length; limit_value=3)

#### Dynamic default values

Recall gotcha with dynamic or mutable default values. Pydantic provides the `default_factory` argument on the `Field` function to cover this use case. This argument expects you to pass a function that will be 
called during model instantiation. Thus, the resulting object will be evaluated at runtime 
each time you create a new object. 

In [39]:
class Post(BaseModel):
    date_created: date = Field(default_factory=lambda: datetime.now())
    tag: Optional[str]

post1 = Post()
time.sleep(3)
post2 = Post()
print((post2.date_created - post1.date_created).total_seconds())

3.005135


The factory function should have no arguments. Moreover, there is no need to set default values in the `Field` functions (which is reasonable).

### Validating email addresses and URLs with Pydantic types

For this to work, you may need `email-validator` which can be installed using `pip`. 

In [43]:
!pip install email-validator

In [70]:
from pydantic import BaseModel, EmailStr, HttpUrl, ValidationError

class User(BaseModel):
    email: EmailStr
    website: HttpUrl

In the following script, we use `ValidationError` as our exception class:

In [57]:
try:
    User(email="user@email,com", website="https://www.example.com")
except ValidationError as e:
    print(str(e))

1 validation error for User
email
  value is not a valid email address (type=value_error.email)


In [60]:
try:
    User(email="user@email.com", website="https://www.example,com")
except ValidationError as e:
    print(str(e))

1 validation error for User
website
  URL host invalid (type=value_error.url.host)


When valid, we get the following parsing for the URL:

In [67]:
User(email="jdoe@example.com", website="https://www.example.com")

User(email='jdoe@example.com', website=HttpUrl('https://www.example.com', scheme='https', host='www.example.com', tld='com', host_type='domain'))

## Creating model variations with class inheritance

Recall in the previous notebook, we saw a case where we needed to 
define two variations of a Pydantic model in order to split between (1) the data we want to 
store in the backend and (2) the data we want to show to the user. This is a common pattern 
in FastAPI: you define one model for **creation**, one for the **response** and one for the **data** to 
store in the database.

In [72]:
class PostCreate(BaseModel):
    title: str
    content: str

class PostPublic(BaseModel):
    id: int
    title: str
    content: str

class PostDB(BaseModel):
    id: int
    title: str
    content: str
    nb_views: int = 0

## Adding custom data validation with Pydantic

### Appying validation at a field level

### Applying validation at an object level

### Applying validation before Pydantic parsing

## Working with Pydantic objects

### Converting an object into a dictionary

### Creating an instance from a sub-class object

### Updating an instance with a partial one